In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.extend([
    'C:/Users/Haakon/OneDrive/Dokumenter/FORSKNING/mastersproject/src/mastersproject',
    'C:/Users/Haakon/OneDrive/Dokumenter/FORSKNING/mastersproject/src/mastersproject/GTS'
])

# Notebook images
from IPython.display import Image

In [3]:
import GTS as gts
# from GTS.ISC_data.isc import ISCData
import numpy as np
import pandas as pd

In [4]:
cls = gts.ISCData()

In [5]:
# Import shear-zone -- borehole data.
sz_bh = cls._shearzone_borehole_data()
sz_bh = sz_bh[sz_bh['depth'].notna()]  # Remove rows with no intersection.
sz_bh.head(3)

,borehole,depth,shearzone
0,INJ1,34.92,S1_1
1,INJ2,30.99,S1_1
2,FBS1,31.09,S1_1


In [6]:
strc = cls._borehole_structure_data().merge(cls._borehole_data(), how='outer', on='borehole', 
                                           suffixes=('_struc', '_bh'), validate='m:1')
# strc['shearzone'] = np.nan
strc1 = strc.loc[:,['depth', 'type', 'borehole']]
strc1.head()

,depth,type,borehole
0,2.35,Minor ductile Shear-zone,FBS1
1,2.99,Minor ductile Shear-zone,FBS1
2,3.27,Foliation,FBS1
3,5.02,Minor ductile Shear-zone,FBS1
4,6.76,Foliation,FBS1


In [7]:
strc1.shape

(591, 3)

In [16]:
# Merge with depth tolerance (atol=0.01)
full_df = pd.merge_asof(strc.sort_values('depth'),
                  sz_bh.sort_values('depth'),
                  by='borehole', 
                  on='depth',
                  tolerance=0.01, 
                  direction='nearest')
df_no_na = full_df[full_df.shearzone.notna()]
# t3 = tt[tt['borehole'] == 'PRP3']

df_filtered = df_no_na[df_no_na.type.isin(['S1 Shear-zone', 'S3 Shear-zone', 'Minor ductile Shear-zone'])]
# df_filtered[df_filtered['borehole'] == 'PRP3']
df_filtered.head(3)

,depth,azimuth_struc,dip,aperture,type,borehole,x,y,z,length,diameter,azimuth_bh,upward_gradient,shearzone
151,15.57,185.46,60.58,308.85,S3 Shear-zone,PRP3,667468.390,158892.660,1733.100,32.33,131,335.01,-25.0,S3_1
199,18.63,165.74,74.19,1244.19,S1 Shear-zone,GEO3,667470.923,158912.008,1732.416,30.10,86,269.96,-47.8,S1_1
216,19.34,166.63,56.55,315.75,S3 Shear-zone,PRP3,667468.390,158892.660,1733.100,32.33,131,335.01,-25.0,S3_2


In [22]:
_mask = full_df.shearzone.notna()
_mask2 = full_df.type.isin(['S1 Shear-zone', 'S3 Shear-zone', 'Minor ductile Shear-zone'])
full_df[_mask & _mask2].shearzone.isna().value_counts()

False    39
Name: shearzone, dtype: int64

In [62]:
def _characterize_shearzones():
    """ Characterize shearzones as Sx_x.
    
    Returns:
    pd.DataFrame: Characterized structures.
    """
    
    # Import shear-zone -- borehole data.
    sz_bh = cls._shearzone_borehole_data()
    sz_bh = sz_bh[sz_bh['depth'].notna()]  # Remove rows with no intersection.
    
    # Import borehole - structure data.
    strc = cls._borehole_structure_data().merge(cls._borehole_data(), how='outer', on='borehole', 
                                               suffixes=('_struc', '_bh'), validate='m:1')
    
    # left merge data structures with simulation-shearzones.
    # Notes:
    # - Absolute tolerance = 0.01
    # - exact match on column = 'borehole'
    _merge = pd.merge_asof(strc.sort_values('depth'),
                  sz_bh.sort_values('depth'),
                  by='borehole', 
                  on='depth',
                  tolerance=0.01, 
                  direction='nearest')
    
    # The above merge includes some nearby 'fractures', etc.
    # All shearzones are classified as 'S1 Shear-zone' or 'S3 Shear-zone'
    # except once: S1_2 intersects GEO3 is of type 'Minor ductile Shear-zone'.
    # With this set of 'type' filters, no spurious shearzone intersections are located.
    ### Below is the complete classified shearzone set.
    _mask_nna = _merge.shearzone.notna()
    _mask_szset = _merge.type.isin(['S1 Shear-zone', 'S3 Shear-zone', 'Minor ductile Shear-zone'])
    _shearzones = _merge[_mask_nna & _mask_szset]
    
    # Step: Get indices indices of non-shearzones:
    _index_other = set(_merge.index) - set(_shearzones.index)
    
    # Set non-shearzones to nan
    _merge.loc[np.array(list(_index_other)), 'shearzone'] = np.nan
    
    return _merge
    
    
    

In [63]:

structures = _characterize_shearzones()

structures

,depth,azimuth_struc,dip,aperture,type,borehole,x,y,z,length,diameter,azimuth_bh,upward_gradient,shearzone
0,1.76,239.29,86.44,1.89,Fracture,SBH3,667468.567,158885.383,1733.960,20.55,101,190.07,4.91,NaN
1,2.11,78.11,21.13,0.00,Fracture,PRP3,667468.390,158892.660,1733.100,32.33,131,335.01,-25.00,NaN
2,2.26,252.34,77.18,1.83,Fracture,SBH3,667468.567,158885.383,1733.960,20.55,101,190.07,4.91,NaN
3,2.35,169.30,57.47,334.61,Minor ductile Shear-zone,FBS1,667466.424,158888.882,1732.782,44.80,101,317.05,-31.92,NaN
4,2.54,134.77,59.79,10.06,Quartz,GEO3,667470.923,158912.008,1732.416,30.10,86,269.96,-47.80,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,47.29,220.12,46.07,0.00,Fracture,PRP1,667466.510,158889.310,1732.640,47.91,131,320.00,-44.00,NaN
587,47.34,193.27,46.28,0.00,Fracture,PRP1,667466.510,158889.310,1732.640,47.91,131,320.00,-44.00,NaN
588,47.36,256.80,28.12,0.00,Fracture,PRP1,667466.510,158889.310,1732.640,47.91,131,320.00,-44.00,NaN
589,47.36,163.26,68.99,0.00,Fracture,FBS2,667470.633,158905.016,1732.725,47.58,101,253.07,-40.46,NaN


In [64]:
structures[structures.shearzone.isna()].shape

(552, 14)

In [65]:
structures[structures.shearzone.notna()].shape

(39, 14)

In [61]:
552+39

591

In [23]:
index_full = set(full_df.index)
index_sz = set(df_filtered.index)

In [24]:
index_other = index_full - index_sz

In [34]:
np.array(list(index_other))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [37]:
full_df.loc[np.array(list(index_other)), 'shearzone'] = np.nan

In [40]:
full_df.tail(20)

,depth,azimuth_struc,dip,aperture,type,borehole,x,y,z,length,diameter,azimuth_bh,upward_gradient,shearzone
571,43.81,161.80,77.90,0.00,Fracture,PRP1,667466.510,158889.310,1732.640,47.91,131,320.00,-44.00,NaN
572,43.93,223.39,24.32,5.01,Fracture,INJ1,667466.231,158888.399,1732.817,44.66,146,309.57,-33.52,NaN
573,43.97,144.17,77.06,0.00,Fracture,FBS2,667470.633,158905.016,1732.725,47.58,101,253.07,-40.46,NaN
574,44.00,150.78,70.76,0.00,Fracture,FBS2,667470.633,158905.016,1732.725,47.58,101,253.07,-40.46,NaN
575,44.03,199.33,22.34,5.15,Quartz,INJ1,667466.231,158888.399,1732.817,44.66,146,309.57,-33.52,NaN
576,44.08,146.85,72.40,557.21,S1 Shear-zone,FBS2,667470.633,158905.016,1732.725,47.58,101,253.07,-40.46,NaN
577,44.13,193.32,24.97,0.00,Fracture,INJ1,667466.231,158888.399,1732.817,44.66,146,309.57,-33.52,NaN
578,44.15,195.43,21.65,0.00,Fracture,PRP2,667466.510,158889.310,1733.120,44.98,131,321.99,-31.99,NaN
579,44.48,182.84,64.21,51.21,Quartz,PRP1,667466.510,158889.310,1732.640,47.91,131,320.00,-44.00,NaN
580,44.84,130.89,21.04,37.45,Quartz,PRP1,667466.510,158889.310,1732.640,47.91,131,320.00,-44.00,NaN


In [27]:
np.isin(np.array(index_full), np.array(index_))

[False]

In [9]:
full_df.shape

(591, 14)

In [10]:
df_filtered.shape

(39, 14)

In [11]:
# The mask can be used to save the index
_mask_sz = strc.loc[:,'type'].isin(['Minor ductile Shear-zone', 
                                    'S1 Shear-zone', 
                                    'S3 Shear-zone']).values

# Original index we have reduced to
sz_idx = strc1.index[_mask_sz]

# Reduced problem: only shear-zone like rows.
sz1 = strc1[_mask_sz]